In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import numpy as np
import plotly.express as px
from sklearn.impute import KNNImputer

In [ ]:
merged_data = pd.read_csv("data/processed/merged_firm_returns.csv")

### Calculate Absolute Ratios

1. <b>Current Ratio</b> : This ratio measures a company's ability to pay off its short-term liabilities with its short-term assets. A higher current ratio indicates a stronger liquidity position.
2. <b>Debt-to-Equity Ratio</b> : This ratio measures the proportion of debt financing to equity financing in a company's capital structure. A higher debt-to-equity ratio indicates a greater reliance on debt financing, which can be a risk factor.
3. <b>Return on Assets (ROA)</b> : This ratio measures the company's ability to generate profits from its assets.
4. <b>Revenue Per Employee</b> : This ratio measures how much revenue a company generates per employee and can be used as a measure of the company's efficiency.
5. <b>Return on Equity (ROE)</b> : This ratio measures the company's ability to generate profits from its shareholders' investments.
6. <b>Asset Turnover Ratio</b> : This ratio measures a company's efficiency in using its assets to generate revenue. A higher asset turnover ratio indicates greater efficiency in asset utilization.
7. <b>Net Income Margin</b> : This ratio measures a company's profitability relative to its revenue. A higher net income margin indicates greater profitability.




In [ ]:
merged_data["Current Ratio"] = (
    merged_data["Total Current Assets"]
    / merged_data["Total Current Liabilities"]
)
merged_data["Debt-to-Equity Ratio"] = (
    merged_data["Total Debt"] / merged_data["Total Equity"]
)
merged_data["Return on Assets"] = (
    merged_data["Net Income - Actual"] / merged_data["Total Assets, Reported"]
)
merged_data["Revenue Per Employee"] = (
    merged_data["Total Revenue"] / merged_data["Company Market Capitalization"]
)
merged_data["Return on Equity"] = (
    merged_data["Net Income - Actual"] / merged_data["Total Equity"]
)
merged_data["Asset Turnover Ratio"] = (
    merged_data["Total Revenue"] / merged_data["Total Assets, Reported"]
)
merged_data["Net Income Margin"] = (
    merged_data["Net Income - Actual"] / merged_data["Total Revenue"]
)

### Outlier Analysis and Treatment

In [ ]:
for i in merged_data.columns:
    box = px.box(merged_data, y=i, width=500, height=500, points="outliers")
    box.show()

In [ ]:
numerical_columns_data = merged_data.copy()
numerical_columns_data.drop(columns=["RIC", "Year", "ESG Score"], inplace=True)

In [ ]:
# Calculate the interquartile range (IQR)
Q1 = numerical_columns_data.quantile(0.25)
Q3 = numerical_columns_data.quantile(0.75)
IQR = Q3 - Q1

# Define the upper and lower bounds for outlier detection
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

# Remove outliers from the dataframe
outliers_removed_dataset = numerical_columns_data[
    ~(
        (numerical_columns_data < lower_bound)
        | (numerical_columns_data > upper_bound)
    ).any(axis=1)
]

In [ ]:
outliers_removed_dataset.head()

In [ ]:
outliers_removed_dataset = outliers_removed_dataset.merge(
    merged_data[["RIC", "Year", "ESG Score"]],
    left_index=True,
    right_index=True,
    how="inner",
)

In [ ]:
outliers_removed_dataset.info()

In [ ]:
outliers_removed_dataset.isna().sum() / len(outliers_removed_dataset) * 100

In [ ]:
merged_data.isnull().sum() / len(merged_data) * 100

In [ ]:
merged_data.describe()

In [ ]:
outliers_removed_dataset.describe()

In [ ]:
for i in outliers_removed_dataset.columns:
    box = px.box(
        outliers_removed_dataset, y=i, width=500, height=500, points="outliers"
    )
    box.show()

In [ ]:
outliers_removed_dataset.to_csv(
    "data/processed/without_outliers_dataset.csv", index=False
)

In [ ]:
merged_data.to_csv("data/processed/with_outliers_dataset.csv", index=False)

### Data Imputation & Final Dataset creation for Model Training

#### 1. Without Outliers

Keep rows only where there is a valid ESG Score

In [ ]:
without_outliers_clean_data = outliers_removed_dataset[
    outliers_removed_dataset["ESG Score"].notna()
]

In [ ]:
subset = list(without_outliers_clean_data.columns)
del subset[-3:]

In [ ]:
subset

In [ ]:
# Define a function to check if at least 90% of values in the subset are null
def check_null_perc(row, subset, null_perc=0.9):
    subset_values = row[subset]
    null_count = subset_values.isna().sum()
    null_perc_subset = null_count / len(subset_values)
    return null_perc_subset >= null_perc

In [ ]:
# Apply the function to remove rows where at least 90% of columns in the subset are null
without_outliers_clean_data = without_outliers_clean_data[
    ~without_outliers_clean_data.apply(
        lambda x: check_null_perc(x, subset), axis=1
    )
]

In [ ]:
len(outliers_removed_dataset), len(without_outliers_clean_data)

In [ ]:
# Normalize the dataset by doing Z score normalization
norm_data_without_outliers = (
    without_outliers_clean_data[subset]
    - without_outliers_clean_data[subset].mean()
) / without_outliers_clean_data[subset].std()

In [ ]:
imputer = KNNImputer(n_neighbors=5, missing_values=np.nan)
imputed_data_without_outliers = imputer.fit_transform(
    norm_data_without_outliers
)

In [ ]:
imputed_without_outliers_df = pd.DataFrame(
    imputed_data_without_outliers,
    columns=subset,
)

In [ ]:
len(imputed_without_outliers_df)

In [ ]:
len(without_outliers_clean_data[["RIC", "Year", "ESG Score"]])

In [ ]:
imputed_without_outliers_df[
    ["RIC", "Year", "ESG Score"]
] = without_outliers_clean_data[["RIC", "Year", "ESG Score"]].reset_index(
    drop=True
)

In [ ]:
imputed_without_outliers_df.describe()

In [ ]:
imputed_without_outliers_df.info()

In [ ]:
imputed_without_outliers_df.isna().sum()

In [ ]:
# Plot a histogram

imputed_without_outliers_df.hist(figsize=(20, 20))

In [ ]:
# export merged dataframe

imputed_without_outliers_df.to_csv(
    "data/model_training_files/imputed_without_outliers.csv"
)

#### 2. With Outliers

Keep rows only where there is a valid ESG Score

In [ ]:
with_outliers_clean_data = merged_data[merged_data["ESG Score"].notna()]

In [ ]:
with_outliers_clean_data.columns

In [ ]:
subset = list(with_outliers_clean_data.columns)
del subset[0]
del subset[9]
del subset[12]

In [ ]:
subset

In [ ]:
# Define a function to check if at least 90% of values in the subset are null
def check_null_perc(row, subset, null_perc=0.9):
    subset_values = row[subset]
    null_count = subset_values.isna().sum()
    null_perc_subset = null_count / len(subset_values)
    return null_perc_subset >= null_perc

In [ ]:
# Apply the function to remove rows where at least 90% of columns in the subset are null
with_outliers_clean_data = with_outliers_clean_data[
    ~with_outliers_clean_data.apply(
        lambda x: check_null_perc(x, subset), axis=1
    )
]

In [ ]:
len(merged_data), len(with_outliers_clean_data)

In [ ]:
merged_data.shape

In [ ]:
with_outliers_clean_data.shape

In [ ]:
# Normalize the dataset by doing Z score normalization
norm_data_with_outliers = (
    with_outliers_clean_data[subset] - with_outliers_clean_data[subset].mean()
) / with_outliers_clean_data[subset].std()

In [ ]:
imputer = KNNImputer(
    n_neighbors=5, missing_values=np.nan, keep_empty_features=True
)
imputed_data_with_outliers = imputer.fit_transform(norm_data_with_outliers)

In [ ]:
norm_data_with_outliers.shape

In [ ]:
imputed_data_with_outliers.shape

In [ ]:
imputed_with_outliers_df = pd.DataFrame(
    imputed_data_with_outliers,
    columns=list(norm_data_with_outliers.columns),
)

In [ ]:
len(imputed_with_outliers_df)

In [ ]:
len(with_outliers_clean_data[["RIC", "Year", "ESG Score"]])

In [ ]:
imputed_with_outliers_df[
    ["RIC", "Year", "ESG Score"]
] = with_outliers_clean_data[["RIC", "Year", "ESG Score"]].reset_index(
    drop=True
)

In [ ]:
imputed_with_outliers_df.describe()

In [ ]:
imputed_with_outliers_df.info()

In [ ]:
imputed_with_outliers_df.isna().sum()

In [ ]:
# Plot a histogram

imputed_with_outliers_df.hist(figsize=(20, 20))

In [ ]:
# export merged dataframe

imputed_with_outliers_df.to_csv(
    "data/model_training_files/imputed_with_outliers.csv"
)